In [1]:
import pandas as pd
import scipy.stats as sts

In [4]:
df = pd.read_csv('socialmobility.csv')
df['combo'] = df['father'] + '_' + df['son']
df = df[['combo', 'count']]
df

,combo,count
0,farm_farm,703
1,farm_unskilled,1478
2,farm_skilled,1430
3,farm_professional,1109
4,unskilled_farm,58
5,unskilled_unskilled,1756
6,unskilled_skilled,1630
7,unskilled_professional,1568
8,skilled_farm,63
9,skilled_unskilled,1453


There are 16 categories, we can start with uniform Dirichlet distribution over $\vec{p}$ with $\vec{\alpha} = (1, \cdots, 1)$.

$$Dirichlet(p_{1}, p_{2}, \cdots, p_{16}) | (1, 1, \cdots, 1)$$

Given that info, we know that the Dirichlet distribution is a conjugate prior for our Multinomial distribution. Therefore the posterior is also a Dirichlet distribution. 
The new alpha is given as $\vec{\alpha_{p\text{post}}} = \vec{\alpha_{\text{prior}}} + \vec{x}$

In [5]:
df['posterior'] = df['count'] + 1
df

,combo,count,posterior
0,farm_farm,703,704
1,farm_unskilled,1478,1479
2,farm_skilled,1430,1431
3,farm_professional,1109,1110
4,unskilled_farm,58,59
5,unskilled_unskilled,1756,1757
6,unskilled_skilled,1630,1631
7,unskilled_professional,1568,1569
8,skilled_farm,63,64
9,skilled_unskilled,1453,1454
